In [2]:
from copy import deepcopy
from IPython.display import clear_output
import time

In [3]:
class Grid:
    def __init__(self, width, height, initial=None):
        self.width = width
        self.height = height
        
        if initial != None:
            self.grid = initial
        else:
            self.grid = []
            
            for i in range(height):
                self.grid.append([None] * self.width)
        
    # row returns the numbers in a row.
    # numbering starts at 0 from the top and moves downwards
    def row(self, row_num):
        return self.grid[row_num]
    
    # col returns the numbers in a column.
    # numbering starts at 0 on the left and moves right
    def col(self, col_num):
        return [self.grid[i][col_num] for i in range(self.height)]
    
    # diag1 returns the first diagonal going across the grid.
    # This means:
    # 1 | O | O
    # O | 2 | O
    # O | O | 3
    # Becomes:
    # [1, 2, 3]
    def diag1(self):
        if self.width != self.height:
            raise DimensionErr
            
        return [self.grid[i][i] for i in range(self.height)]
    
    # diag2 returns the second diagonal going across the grid.
    # This means:
    # O | O | 3
    # O | 2 | O
    # 1 | O | O
    # Becomes:
    # [1, 2, 3]
    def diag2(self):
        if self.width != self.height:
            raise DimensionErr
            
        return [self.grid[self.height - i - 1][i] for i in range(self.height)]
    
    # update updates the cell at (x, y), starting from the top (y) left (x).
    def update(self, x, y, val):
        self.grid[y][x] = val
        
    # first_none returns the coordinate of the first None filled cell, moving left to right, top to bottom.
    def first_none(self):
        for y in range(self.height):
            for x in range(self.width):
                if self.grid[y][x] == None:
                    return (x, y)
                
        return None
    
    # magic returns true if the columns, rows and diagonals all add up to the same
    def magic(self, should_equal):
        for i in range(self.height):
            if sum(self.row(i)) != should_equal:
                return False
            
        for i in range(self.width):
            if sum(self.col(i)) != should_equal:
                return False

        return sum(self.diag1()) == sum(self.diag2()) == should_equal
    
    # partial_magic returns true if the columns, rows and diagonals that don't include None all add up to the same
    def partial_magic(self, should_equal):
        for i in range(self.height):
            row = self.row(i)
            if None not in row and sum(row) != should_equal:
                return False
            
        for i in range(self.width):
            col = self.col(i)
            if None not in col and sum(col) != should_equal:
                return False
            
        diag1 = self.diag1()
        if None not in diag1 and sum(diag1) != should_equal:
            return False
        
        diag2 = self.diag2()
        if None not in diag2 and sum(diag2) != should_equal:
            return False
        
        return True
    
    def __str__(self):
        str_rows = []
        
        for row in self.grid:
            str_rows.append(" | ".join(["_" if x == None else str(x) for x in row]))
            
        return ("\n" + "-" * len(str_rows[0]) + "\n").join(str_rows)
    
    def __repr__(self): return self.__str__()

In [4]:
magic3x3 = Grid(
    3, 3, 
    [
        [1,0,2],
        [2,1,0],
        [0,2,1]
    ]
)
magic3x3.magic(3)

True

In [5]:
magic4x4 = Grid(
    4, 4,
    [
        [2, 0, 3, 1],
        [1, 3, 0, 2],
        [0, 2, 1, 3],
        [3, 1, 2, 0],
    ],
)
magic4x4.magic(6)

True

In [6]:
magic5x5 = Grid(
    5, 5,
    [
        [0, 1, 2, 3, 4],
        [3, 4, 0, 1, 2],
        [1, 2, 3, 4, 0],
        [4, 0, 1, 2, 3],
        [2, 3, 4, 0, 1],
    ],
)
magic5x5.magic(10)

True

In [7]:
magic5x5

0 | 1 | 2 | 3 | 4
-----------------
3 | 4 | 0 | 1 | 2
-----------------
1 | 2 | 3 | 4 | 0
-----------------
4 | 0 | 1 | 2 | 3
-----------------
2 | 3 | 4 | 0 | 1

In [126]:
# weak_solve solves the magic square problem using a backtracking algorithm
# weak magic squares can have duplicates in the rows, columns and diagonals.
def weak_solve(grid, inventory, magic):
    first_none = grid.first_none()
    
    if first_none == None:
        if grid.magic(magic):
            return grid
        else:
            return None
    
    x, y = first_none
    candidates = [num for num in inventory.keys() if inventory[num] > 0]
    
    for candidate in candidates:
        copied_grid = deepcopy(grid)
        copied_grid.grid[y][x] =  candidate
        
        # if adding this candidate means that a row/column/diagonal is not magic, then why try it?
        if not grid.partial_magic(magic):
            continue
        
        copied_inventory = deepcopy(inventory)
        copied_inventory[candidate] -= 1

        solved = weak_solve(copied_grid, copied_inventory, magic)
        
        if solved != None:
            return solved
    
# moderate_solve solves the magic square problem using a backtracking algorithm
# moderate magic squares can have duplicates in the diagonals, but not rows and columns
def moderate_solve(grid, inventory, magic):
    first_none = grid.first_none()
    
    if first_none == None:
        if grid.magic(magic):
            return grid
        else:
            return None
    
    x, y = first_none
    candidates = [num for num in inventory.keys() if inventory[num] > 0]
    
    for candidate in candidates:
        copied_grid = deepcopy(grid)
        copied_grid.grid[y][x] =  candidate
        
        # if adding this candidate means that a row/column/diagonal is not magic, then why try it?
        # if we have duplicates in the row or column we can skip it
        if not grid.partial_magic(magic) or candidate in grid.row(y) or candidate in grid.col(x):
            continue
        
        copied_inventory = deepcopy(inventory)
        copied_inventory[candidate] -= 1

        solved = moderate_solve(copied_grid, copied_inventory, magic)
        
        if solved != None:
            return solved
        
# strong_solve solves the magic square problem using a backtracking algorithm
# strong magic squares can't have duplicates in the rows, columns or diagonals.
def strong_solve(grid, inventory, magic):
    first_none = grid.first_none()
    
    #print(grid)
    #time.sleep(0.5)
    #clear_output()
    
    if first_none == None:
        if grid.magic(magic):
            return grid
        else:
            return None
    
    x, y = first_none
    candidates = [num for num in inventory.keys() if inventory[num] > 0]
    
    for candidate in sorted(candidates):
        copied_grid = deepcopy(grid)
        copied_grid.grid[y][x] =  candidate
        
        # if adding this candidate means that a row/column/diagonal is not magic, then why try it?
        # if we have duplicates in the row or column we can skip it
        if not grid.partial_magic(magic) or candidate in grid.row(y) or candidate in grid.col(x):
            continue
            
        # prevent any duplicates in the diagonals
        if y == x and candidate in grid.diag1():
            continue
            
        if grid.height - y - 1 == x and candidate in grid.diag2():
            continue
        
        copied_inventory = deepcopy(inventory)
        copied_inventory[candidate] -= 1

        solved = strong_solve(copied_grid, copied_inventory, magic)
        
        if solved != None:
            return solved

In [127]:
# general solves the general strong nxn square problem
def general(n):
    grid = Grid(n, n)
    return strong_solve(grid, {i: n for i in range(n)}, n*(n-1)//2)

In [128]:
general(3)

In [129]:
general(4)

0 | 1 | 2 | 3
-------------
2 | 3 | 0 | 1
-------------
3 | 2 | 1 | 0
-------------
1 | 0 | 3 | 2

In [130]:
strong_solve(
    Grid(
        5, 5,
        [
            [0, 1, 2, 3, 4],
            [None, 2, None, None, None],
            [None, None, None, None, None],
            [None, None, None, None, None],
            [None, None, None, None, None],
        ]
    ),
    {0: 4, 1: 4, 2: 3, 3: 4, 4: 4},
    10
)

0 | 1 | 2 | 3 | 4
-----------------
4 | 2 | 0 | 1 | 3
-----------------
1 | 4 | 3 | 2 | 0
-----------------
3 | 0 | 1 | 4 | 2
-----------------
2 | 3 | 4 | 0 | 1

In [131]:
general(5)

0 | 1 | 2 | 3 | 4
-----------------
1 | 3 | 4 | 2 | 0
-----------------
4 | 2 | 1 | 0 | 3
-----------------
2 | 0 | 3 | 4 | 1
-----------------
3 | 4 | 0 | 1 | 2

In [132]:
general(6)

0 | 1 | 2 | 3 | 4 | 5
---------------------
1 | 2 | 0 | 5 | 3 | 4
---------------------
4 | 3 | 5 | 0 | 2 | 1
---------------------
3 | 0 | 1 | 4 | 5 | 2
---------------------
5 | 4 | 3 | 2 | 1 | 0
---------------------
2 | 5 | 4 | 1 | 0 | 3

In [ ]:
general(7)

In [ ]:
general(8)

In [133]:
def complete(grid):
    nums = [num for row in grid for num in row]
    
    width = len(grid[0])
    height = len(grid)
    
    inventory = {i: width-nums.count(i) for i in range(width)}
    
    solved = strong_solve(Grid(width, height, grid), inventory, sum(range(width)))
    
    return "No solutions" if solved == None else solved

In [142]:
complete([
    [0, 1, 2, 3, 4],
    [None, None, None, None, None],
    [None, None, None, None, None],
    [None, None, None, None, None],
    [None, None, None, None, None]
])

0 | 1 | 2 | 3 | 4
-----------------
1 | 3 | 4 | 2 | 0
-----------------
4 | 2 | 1 | 0 | 3
-----------------
2 | 0 | 3 | 4 | 1
-----------------
3 | 4 | 0 | 1 | 2

In [161]:
complete([
    [0,3,2,1],
    [None, None, None, None],
    [1, None, None, None],
    [None, None, None, None],
])

0 | 3 | 2 | 1
-------------
2 | 1 | 0 | 3
-------------
1 | 2 | 3 | 0
-------------
3 | 0 | 1 | 2